In [1]:
import json

In [2]:
from time import sleep

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


class MediaDownloader():
    def __init__(self, url):
        self.url = url
        self.driver = self.driver_init()
        
    def driver_init(self):
        chrome_path = ChromeDriverManager().install()
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36"

        options = webdriver.ChromeOptions()
        options.headless = True
        options.add_argument(f'user-agent={user_agent}')
        options.add_argument("--window-size=1920,1080")
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--allow-running-insecure-content')
        options.add_argument("--disable-extensions")
        options.add_argument("--proxy-server='direct://'")
        options.add_argument("--proxy-bypass-list=*")
        options.add_argument("--start-maximized")
        options.add_argument('--disable-gpu')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--no-sandbox')
        
        service = Service(executable_path=chrome_path)
        driver = webdriver.Chrome(service=service, options=options) #, options=options
        
        return driver
        
    def scraping_data(self):
        self.driver.get('https://savefrom.net/')
        
        sleep(1)
        self.driver.find_element(By.NAME, 'sf_url').send_keys(self.url)
        self.driver.find_element(By.NAME, "sf_submit").click()

        try:
            ele = WebDriverWait(self.driver, 15).until( #using explicit wait for 10 seconds
                EC.presence_of_element_located((By.XPATH, "//*[@id='sf_result']/div/div[1]/div[2]/div[2]")) #checking for the element with 'h2'as its CSS
            )
            
            page_source = self.driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            
        except:
            soup = None
            print("Timeout Exception: Page did not load within 10 seconds.")
        finally:
            self.driver.close()
            return soup
        
    
    def json(self):
        soup = self.scraping_data()
        
        if soup is not None:
            
            metas = soup.find_all("div", {"class": "info-box"})
            
            full_dict = []  # Create an empty list

            for meta in metas:
                url= meta.find('a').get("href") if meta is not None else ''
                title = meta.find("div", {"class": "row title"}).get_text() if meta is not None else ''
                type = meta.find('a').get_text() if meta is not None else ''
                if type != 'Download':
                    type = type.split(" ")[2]
                elif type == 'Download':
                    type = 'MP4'
                
                my_dict = {
                    "title": title,
                    "type": type,
                    "url": url
                }  # Create a dictionary

                full_dict.append(my_dict)  # Append the dictionary to the list
                
            
            return full_dict
        

In [3]:
r = MediaDownloader('https://www.instagram.com/p/CqFhVU5Loow/')
print(json.dumps(r.json(), indent=2))

C:\Users\Ryas\AppData\Local\Temp\ipykernel_3424\1844620286.py:22: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


[
  {
    "title": "Rahajeng rahina nyepi   \ud83e\udef6",
    "type": "JPG",
    "url": "https://instagram.fsub9-1.fna.fbcdn.net/v/t51.2885-15/336958004_3522264818004012_3866986351418766689_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=instagram.fsub9-1.fna.fbcdn.net&_nc_cat=100&_nc_ohc=q6luQ5Tn8_EAX9XeZA4&edm=AP_V10EBAAAA&ccb=7-5&oh=00_AfBxweg0NO-MwEX_GlF__BgKy-zf8vrswrw9XDoojBGENA&oe=6498D0CC&_nc_sid=8721cf&dl=1"
  }
]


In [4]:
r = MediaDownloader('https://www.instagram.com/p/Ctgg8EZhcIe/')
print(json.dumps(r.json(), indent=2))

C:\Users\Ryas\AppData\Local\Temp\ipykernel_3424\1844620286.py:22: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


[
  {
    "title": "Ayoo ramaikan Kapal Api Group Indonesia Open 2023 karena masih sampe tgl 18 Juni minggu ini!! Yuk dateng dan dukung Atlet Badminton Indonesia   Serukan semangatmu bersama Kopi Kapal Api!@kapalapi_id#SerukanSemangatmu",
    "type": "MP4",
    "url": "https://media.sf-converter.com/get?__sig=qYRQwxw1tEeM6_fIgLFC7A&__expires=1687408846&uri=https%3A%2F%2Finstagram.fgye1-2.fna.fbcdn.net%2Fv%2Ft66.30100-16%2F10000000_3652362781712602_8731603251370293570_n.mp4%3F_nc_ht%3Dinstagram.fgye1-2.fna.fbcdn.net%26_nc_cat%3D110%26_nc_ohc%3Dt6mtPuj1dLIAX_5SWCV%26edm%3DAP_V10EBAAAA%26ccb%3D7-5%26oh%3D00_AfAJYn473CYaBKbnPYZtV7zehTVEmy517xwyXzHmM910iw%26oe%3D64954FCF%26_nc_sid%3D8721cf%26dl%3D1&filename=Ayoo%20ramaikan%20Kapal%20Api%20Group%20Indonesia%20Open%202023%20karena%20masih%20sampe%20tgl%2018%20Juni%20minggu%20ini!!%20Yuk%20dateng%20dan%20dukung%20Atlet%20Badminton%20Indonesia%20%20%20Serukan%20semangatmu%20bersama%20Kopi%20Kapal%20Api!%40kapalapi_id%23SerukanSemangatmu.mp4&ua=

In [5]:
r = MediaDownloader('https://www.instagram.com/p/Cs8SqNqrW3_/')
print(json.dumps(r.json(), indent=2))

C:\Users\Ryas\AppData\Local\Temp\ipykernel_3424\1844620286.py:22: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


[
  {
    "title": "Spirit Doll tayang hari ini di bioskop! Dapet salam nih dari Dara Lazuardi, katanya see you soon!",
    "type": "JPG",
    "url": "https://instagram.fakx4-1.fna.fbcdn.net/v/t51.2885-15/350494926_772144671114721_2802072320226141595_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=instagram.fakx4-1.fna.fbcdn.net&_nc_cat=101&_nc_ohc=53RAJsIPqSMAX80h878&edm=AP_V10EBAAAA&ccb=7-5&oh=00_AfAOcqVFe7ZdyC89kZ_eiRibLZxFwUvkTXXCzD7OS9IsVQ&oe=6495D5CD&_nc_sid=8721cf&dl=1"
  },
  {
    "title": "Spirit Doll tayang hari ini di bioskop! Dapet salam nih dari Dara Lazuardi, katanya see you soon!",
    "type": "JPG",
    "url": "https://instagram.fakx4-1.fna.fbcdn.net/v/t51.2885-15/350499564_262568716290740_8197150143586452704_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=instagram.fakx4-1.fna.fbcdn.net&_nc_cat=108&_nc_ohc=kG7aV0GQodoAX-svCRI&edm=AP_V10EBAAAA&ccb=7-5&oh=00_AfCXBBRH_EvV4Z8CauoX1iGFTOfSPvu1y7TXxWp1ovy9og&oe=64963BE3&_nc_sid=8721cf&dl=1"
  },
  {
    "title": "Spirit Doll tayang hari in

In [6]:
r = MediaDownloader('https://twitter.com/PopBase/status/1671555535873937408')
print(json.dumps(r.json(), indent=2))

C:\Users\Ryas\AppData\Local\Temp\ipykernel_3424\1844620286.py:22: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


[
  {
    "title": "A documentary on the missing Titanic tourist submarine titled \u2018Titanic Sub: Lost at Sea\u2019 will air this Thursday on the UK\u2019s Channel 5.",
    "type": "JPG",
    "url": "https://pbs.twimg.com/media/FzKO5mxaUAEUFCH.jpg"
  },
  {
    "title": "A documentary on the missing Titanic tourist submarine titled \u2018Titanic Sub: Lost at Sea\u2019 will air this Thursday on the UK\u2019s Channel 5.",
    "type": "JPG",
    "url": "https://pbs.twimg.com/media/FzKO5mzaIAA2Ryf.jpg"
  }
]


In [7]:
r = MediaDownloader('https://twitter.com/FabrizioRomano/status/1671648252486119424')
print(json.dumps(r.json(), indent=2))

C:\Users\Ryas\AppData\Local\Temp\ipykernel_3424\1844620286.py:22: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


[
  {
    "title": "Understand Manchester United will bid again for Mason Mount. Personal terms are 100% agreed and sealed. \ud83d\udea8\ud83d\udd34 #MUFC  Mount plans not to sign new deal at Chelsea.  Man Utd want quick resolution \u2014 and won\u2019t make \u2018crazy\u2019 bids.  Chelsea now flexible; \u00a360/65m enough to get it done.",
    "type": "JPG",
    "url": "https://pbs.twimg.com/media/FzLjOVfX0AAHbuI.jpg"
  }
]


In [8]:
r = MediaDownloader('https://twitter.com/NIJISANJI_World/status/1671669368646942721')
print(json.dumps(r.json(), indent=2))

C:\Users\Ryas\AppData\Local\Temp\ipykernel_3424\1844620286.py:22: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


[
  {
    "title": "\u3010New VTuber Debut Announcement\u3011  NIJISANJI EN announces the debut of 3 new VTubers: @YuQWilson @Tyrant_Vanta & @VezaliusBandage !  \ud83d\udd3bDebut teaser:  \ud83d\udd3bDebut program: \ud83d\udd5a June 25, 11 AM JST | 2 AM GMT | June 24, 7 PM PST For the\u2026",
    "type": "MP4",
    "url": "https://video.twimg.com/ext_tw_video/1671662550487011329/pu/vid/1280x720/BGPx9PsXrayp3Eth.mp4?tag=12"
  }
]


In [9]:
r = MediaDownloader('https://www.youtube.com/watch?v=U_k5xNlFofw&pp=ygULaW50cm8gbXVzaWM%3D')
print(json.dumps(r.json(), indent=2))

C:\Users\Ryas\AppData\Local\Temp\ipykernel_3424\1844620286.py:22: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


[
  {
    "title": "Cinematic Intro Music Template FREE",
    "type": "MP4",
    "url": "https://rr1---sn-apcxgutj5cax2g-npbe.googlevideo.com/videoplayback?expire=1687412110&ei=LomTZLSFKZiAy_sPguiZ0AQ&ip=2605%3Aba00%3Ae117%3Afe6e%3A1807%3Af342%3A9f3f%3A27bb&id=o-AJ0sEwAtuJ67CJFrZKhwj9uvp4eL-aHlMeo02lwNOekX&itag=22&source=youtube&requiressl=yes&mh=ki&mm=31%2C29&mn=sn-apcxgutj5cax2g-npbe%2Csn-hp57ynl6&ms=au%2Crdu&mv=m&mvi=1&pl=53&initcwndbps=2063750&spc=qEK7B7DIwI2EFg6GxNhg6y4caHAEbxxwBrF5sppOHQ&vprv=1&svpuc=1&mime=video%2Fmp4&ns=Th3HRJGEBcrfl7TDQlfHO4kN&cnr=14&ratebypass=yes&dur=111.875&lmt=1673892975548832&mt=1687390165&fvip=4&fexp=24007246%2C24362687&c=WEB&txp=5311224&n=v7Y6kfxbqVi_4Q&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cspc%2Cvprv%2Csvpuc%2Cmime%2Cns%2Ccnr%2Cratebypass%2Cdur%2Clmt&sig=AOq0QJ8wRQIhAJZ5ebJwJtdjX6Pt4KiNtqa3aW_w3GHpck2RBpae3CvPAiBdRQpUeJ4t2TR-WTvZSWswZsFoOPci6QgxNmG-zR7cwQ%3D%3D&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AG3C_xA

In [10]:
r = MediaDownloader('https://www.youtube.com/shorts/yKRABoMmgWM')
print(json.dumps(r.json(), indent=2))

C:\Users\Ryas\AppData\Local\Temp\ipykernel_3424\1844620286.py:22: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


[
  {
    "title": "JIAAKH mantan kok bekas bekasan~ #shorts #tonightshowpremiere",
    "type": "MP4",
    "url": "https://rr1---sn-4g5lznl6.googlevideo.com/videoplayback?expire=1687429296&ei=T8yTZJLaO8Gg6dsPx-2ciAs&ip=91.90.122.23&id=o-AMf0sf6f2ei42m8zS2cJtwd6YqJ5rnlqfM54nAHzy3zC&itag=22&source=youtube&requiressl=yes&mh=Kq&mm=31%2C29&mn=sn-4g5lznl6%2Csn-4g5edn6y&ms=au%2Crdu&mv=m&mvi=1&pl=24&initcwndbps=768750&bui=AZ6lgGG3mpX-Ey3YXeUkMNSOEDpVEzQTRyPoNbgApkQ5AapLBK0Sz4_o-llsDXvnvNz3cVZFEKiD7d8uazgKL6IQAz6lsaH9&spc=qEK7B73HI5zIiiK_t0n9x6tCKehT_pZCaYyLPOgV5A&vprv=1&svpuc=1&mime=video%2Fmp4&ns=-PgFx1i4GwesP6q8QVrKTSQN&cnr=14&ratebypass=yes&dur=38.452&lmt=1679296447385970&mt=1687407193&fvip=1&fexp=24007246&beids=24350017&c=WEB&txp=6318224&n=ul_bKI1ivNmbEQ&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cbui%2Cspc%2Cvprv%2Csvpuc%2Cmime%2Cns%2Ccnr%2Cratebypass%2Cdur%2Clmt&sig=AOq0QJ8wRAIgSPZOLia7w-qBESyW09pdIXePCRvzT6RScf4kZ-AiQ9ECIEFJvkzGwmBkYvCzKZ7XNDv8okpEVV-GV1NC2jHpDjCA&lspar

In [11]:
r = MediaDownloader('https://www.tiktok.com/@vilmeifanbsa/video/7247169789578300677?is_from_webapp=1&sender_device=pc')
print(json.dumps(r.json(), indent=2))

C:\Users\Ryas\AppData\Local\Temp\ipykernel_3424\1844620286.py:22: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


[
  {
    "title": "1 MALL DIBIKIN KAGET SEMUA",
    "type": "MP4",
    "url": "https://tt.sf-converter.com/get?payload=tlZ1Sk1ujpfRVw9AVEI-sFlbUqCIdcQfQnKCbaY0mMGlZlVfZQoZtsyUwuuFrvd3mb3p4Ipr0M1MpQ2rplk871o5N9knDS7EtiOMY7458dgt8sDgvIOl5emNFvv1BQCJJHgIeacVyHual5MFrF4Vl~UQgvkx9F1Rp1nvoowYr5Q~jkQ7vNH4YbBCyuPqByLBWdtb1i4MZkTrydiD6n~ilnty~~m7FUhNTpEqkPHfpYSvuCPKf2S3hjMm4dKvyCMZZTc-EufnBGqrCHOCfRhyvPKF4mKHrlKISykoLKuN6T77Es0S-LSzCkhEh715lkVEWTRR9Q9BEDMO9wikoy8ZW6dNZtjpFWyHm0nqm-n6I~qM0krxIfmMF1tdxsbtrS5qFxduvoJT-KvymdeG9Jmy-qJM~OwZEu6j0q9onWmkW4kKOh7OMnSe7iiTiygcsLqMX80DX5BHB6YXzxmRHil1aUVxv9WZeKo4ldBVmACgyb4pfgCSe6HFx0FW4QI0Rrj-wpZUkNCALU7v~Ez08ITd9JIYnwwE9gJua9RGjCGM3WbaPrKrm-8fw2airZ__*ca8bba6123099aba*1*1687407708"
  }
]


In [12]:
r = MediaDownloader('https://www.tiktok.com/@valezka_07/video/7243319359983176966?is_from_webapp=1&sender_device=pc')
print(json.dumps(r.json(), indent=2))

C:\Users\Ryas\AppData\Local\Temp\ipykernel_3424\1844620286.py:22: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


[
  {
    "title": "Duo kulbet",
    "type": "MP4",
    "url": "https://tt.sf-converter.com/get?payload=tlZ1SNbu4xfBVw-AAQn9NKxbXLHzkgbKfcEKWdgZg-LcmHt5V9qv4DAU~9o3dEIrpWaJVYhrj-hR2ddBlQUa1njD6yFEkRLN5UdAIMzsaUdRXtkOSdvrzpCjZFncHcv3pDvN4RqBcLdvbvZHB6lw0QXkWUFsuk0STYejxpq4qFLnmvXKUKfB61Rpkaw2QUGg~Md124QdJTJM3MU6hN-dhPbtPJ0xobl-JFOBVRfSdC81BZOK694kPtPEfReK4PyO-VOBE4DfRqyqZfBvaaEcqccswaXoCfCIOjSdQY-346W-TLfuXG19Kp3v1hMVnAjOpsM9E5IrAO-UwrToy8~Yyluz47CaEFpuZFn-2JHMr1w5thdl3Uj3h53ieQqw2IH-g26X43DtlpFMpUnPmJNWPW0LhMupibyT4e7A3b~8C1~aRjeT6Gs14o0TGb2i34hdS~m0XBUCKeOzZIasCK3CqhOkVS0FhaCcr9ftvwgf4pCevvnsUp~POAT2dckgV7lWbHhh17HkrOR88Nn233Ms6SwtEFERZY5m~-FmKD2Tvz__*34307fe273220540*1*1687407720"
  }
]


In [13]:
r = MediaDownloader('https://www.facebook.com/reel/537369971772656')
print(json.dumps(r.json(), indent=2))

C:\Users\Ryas\AppData\Local\Temp\ipykernel_3424\1844620286.py:22: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


[
  {
    "title": "HUMOR HERP PAIDPROMOTE on Reels | ",
    "type": "MP4",
    "url": "https://scontent-hkg4-1.xx.fbcdn.net/v/t50.33967-16/354087150_1086286175546186_1506721674466659806_n.mp4?_nc_cat=100&ccb=1-7&_nc_sid=985c63&efg=eyJybHIiOjE4MTgsInJsYSI6MTAyNCwidmVuY29kZV90YWciOiJ4cHZfaGRfcHJvZ3Jlc3NpdmUifQ%3D%3D&_nc_ohc=zXDHJs21DI0AX8q8I9Q&rl=1818&vabr=1212&_nc_ht=scontent-hkg4-1.xx&oh=00_AfB2lwSBS_F5bb8AXbenZCnnsllDjGRyr4ph5NGwzu0e5A&oe=64984E4E&dl=1"
  }
]


In [20]:
r = MediaDownloader('https://www.facebook.com/photo/?fbid=285494414007693&set=a.186996913857444')
print(json.dumps(r.json(), indent=2))

C:\Users\Ryas\AppData\Local\Temp\ipykernel_3424\1844620286.py:22: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


[
  {
    "title": "Photo",
    "type": "JPG",
    "url": "https://scontent-arn2-1.xx.fbcdn.net/v/t39.30808-6/355474511_285494500674351_85709678798895839_n.jpg?_nc_cat=102&ccb=1-7&_nc_sid=8bfeb9&_nc_ohc=56-TKNcUlYYAX-uIX5j&_nc_ht=scontent-arn2-1.xx&oh=00_AfCZ7WePWOgaDNWx14RkS9TFBOL9pxglFuke6vjE0w9iRg&oe=6497D73B&dl=1"
  }
]


In [15]:
r = MediaDownloader('https://www.facebook.com/100076414081234/videos/723503226026650')
print(json.dumps(r.json(), indent=2))

C:\Users\Ryas\AppData\Local\Temp\ipykernel_3424\1844620286.py:22: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


[
  {
    "title": "Video 723503226026650",
    "type": "MP4",
    "url": "https://video-mia3-1.xx.fbcdn.net/v/t39.25447-2/336650222_581867480566355_5889654181237998456_n.mp4?_nc_cat=106&vs=d3da39f8730ebadc&_nc_vs=HBksFQAYJEdPN2ZFQlJUU255VU5CRUNBSGpEbGVwRlFieFJibWRqQUFBRhUAAsgBABUAGCRHRVhGUnhOLU45anFIa01DQUNJQ1BHRmgzWkJGYnY0R0FBQUYVAgLIAQBLB4gScHJvZ3Jlc3NpdmVfcmVjaXBlATENc3Vic2FtcGxlX2ZwcwAQdm1hZl9lbmFibGVfbnN1YgAgbWVhc3VyZV9vcmlnaW5hbF9yZXNvbHV0aW9uX3NzaW0AKGNvbXB1dGVfc3NpbV9vbmx5X2F0X29yaWdpbmFsX3Jlc29sdXRpb24AHXVzZV9sYW5jem9zX2Zvcl92cW1fdXBzY2FsaW5nABFkaXNhYmxlX3Bvc3RfcHZxcwAVACUAHIwXQAAAAAAAAAAREQAAACai4MTdq%2BaBCRUCKAJDMxgLdnRzX3ByZXZpZXccF0AuiTdLxqfwGDRkYXNoX2k0bGl0ZWJhc2ljX3Bhc3N0aHJvdWdoYWxpZ25lZF9ocTFfZnJhZ18yX3ZpZGVvEgAYGHZpZGVvcy52dHMuY2FsbGJhY2sucHJvZDgSVklERU9fVklFV19SRVFVRVNUGwqIFW9lbV90YXJnZXRfZW5jb2RlX3RhZwZvZXBfaGQTb2VtX3JlcXVlc3RfdGltZV9tcwEwDG9lbV9jZmdfcnVsZQd1bm11dGVkE29lbV9yb2lfcmVhY2hfY291bnQGMTUyNjE0EW9lbV9pc19leHBlcmltZW50AAxvZW1fdmlkZW9faWQPNzIzNTAzMjI2MDI2NjUw

<h1>TEST</h1>

In [16]:
# import json

# from time import sleep

# from bs4 import BeautifulSoup
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from webdriver_manager.chrome import ChromeDriverManager

# chrome_path = ChromeDriverManager().install()
# user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36"

# options = webdriver.ChromeOptions()
# options.headless = True
# options.add_argument(f'user-agent={user_agent}')
# options.add_argument("--window-size=1920,1080")
# options.add_argument('--ignore-certificate-errors')
# options.add_argument('--allow-running-insecure-content')
# options.add_argument("--disable-extensions")
# options.add_argument("--proxy-server='direct://'")
# options.add_argument("--proxy-bypass-list=*")
# options.add_argument("--start-maximized")
# options.add_argument('--disable-gpu')
# options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--no-sandbox')

# service = Service(executable_path=chrome_path)
# driver = webdriver.Chrome(service=service, options=options) #, options=options

In [17]:
# driver.get('https://savefrom.net/')
        
# sleep(1)
# driver.find_element(By.NAME, 'sf_url').send_keys('https://www.instagram.com/stories/anyageraldine/3129802287383223883/')
# driver.find_element(By.NAME, "sf_submit").click()

# try:
#     ele = WebDriverWait(driver, 10).until( #using explicit wait for 10 seconds
#         EC.presence_of_element_located((By.XPATH, "//*[@id='sf_result']/div/div[1]/div[2]/div[2]")) #checking for the element with 'h2'as its CSS
#     )
    
#     page_source = driver.page_source
#     soup = BeautifulSoup(page_source, 'html.parser')

#     metas = soup.find_all("div", {"class": "info-box"})

#     for meta in metas:
#         print(meta.find('a').get("href") if meta is not None else '')
#         print(meta.find("div", {"class": "row title"}).get_text() if meta is not None else '')
#         print(meta.find('a').get_text().split(" ")[2] if meta is not None else '')
# except:
#     page_source = None
#     print("Timeout Exception: Page did not load within 10 seconds.")

In [18]:
# meta = soup.find("div", {"class": "row title"})
# title = meta.get_text() if meta is not None else ''

# meta = soup.find("div", {"class": "row duration"})
# duration = meta.get_text() if meta is not None else ''

# meta = soup.find_all('a', {"data-type" : True})
# type = meta[0].get("data-type") if meta is not None else ''

# meta = soup.find_all('a', {"download" : True})
# link = meta[0].get("href") if meta is not None else ''

# obj = {
#     "title": title,
#     "duration": duration,
#     "type": type,
#     "url": link
# }

# print(json.dumps(obj, indent=2))

In [19]:
# my_list = []  # Create an empty list

# my_dict = {
#     "title": 'title',
#     "duration": 'duration',
#     "type": 'type',
#     "url": 'link'
# }  # Create a dictionary

# my_list.append(my_dict)  # Append the dictionary to the list
# my_list.append(my_dict)  # Append the dictionary to the list
# my_list.append(my_dict)  # Append the dictionary to the list

# print(my_list)  # Output: [{'name': 'John', 'age': 30}]

# print(json.dumps(my_list, indent=2))